# ortools & 数独

In [1]:
from ortools.linear_solver import pywraplp

## 整数规划

相应的api的执行逻辑和scip基本一致

In [7]:
matrix = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

N = 9
# litte matrix shape
n = 3
# 填入的数字
constants = [e for e in range(1, N + 1)]

In [77]:
# 也可以使用其他的求解器

solver = pywraplp.Solver.CreateSolver('SCIP')

In [78]:
# 操作的api基本和pyscipot一致

variables = {}
for r in range(N):
    for c in range(N):
        for k in constants:
            key = (r, c, k)
            variables[key] = solver.BoolVar(name = '_'.join(str(e) for e in key))

In [79]:
for r in range(N):
    for c in range(N):
        if matrix[r][c] != 0:
            constrain = variables[(r, c, matrix[r][c])] == 1
            solver.Add(constrain)

In [80]:
for r in range(N):
    for c in range(N):
        constrain = sum(variables[(r, c, k)] for k in constants) == 1
        solver.Add(constrain)

In [81]:
for i in range(N):
    for k in constants:
        # 行约束
        r_constrain = sum(variables[i, c, k] for c in range(N)) == 1
        solver.Add(r_constrain)
        # 列约束
        c_constrain = sum(variables[r, i, k] for r in range(N)) == 1
        solver.Add(c_constrain)

In [82]:
# 约束4, 子宫格约束              
for row in range(n):
    for col in range(n):
        for k in constants:
            constrain = sum(variables[r + n * row, c + n * col, k] for r in range(n) for c in range(n)) == 1
            solver.Add(constrain)

In [83]:
status = solver.Solve()

In [87]:
if status == pywraplp.Solver.OPTIMAL:
    print('sudoku solution:')
    # 预先构造一个数独容器用于存放解析结果
    solution = [[0 for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            solution[i][j] = sum(k * int(variables[i, j, k].solution_value()) for k in constants)
    # 绘制数独结果
    sudo_data = (''.join(f'{"| " if c % 3 == 0 else " "}{solution[r][c]}{" " if matrix[r][c] == 0 else "*"}' for c in range(N)) + '|' + ('\n'+ '-' * 31 if (r + 1) % 3 == 0 and r != N - 1 else '') for r in range(N))
    print('\n'.join(sudo_data))
else: print('warning: no solution')

sudoku solution:
| 8* 1  2 | 7  5  3 | 6  4  9 |
| 9  4  3*| 6* 8  2 | 1  7  5 |
| 6  7* 5 | 4  9* 1 | 2* 8  3 |
-------------------------------
| 1  5* 4 | 2  3  7*| 8  9  6 |
| 3  6  9 | 8  4* 5*| 7* 2  1 |
| 2  8  7 | 1* 6  9 | 5  3* 4 |
-------------------------------
| 5  2  1*| 9  7  4 | 3  6* 8*|
| 4  3  8*| 5* 2  6 | 9  1* 7 |
| 7  9* 6 | 3  1  8 | 4* 5  2 |


## 约束规划

关于sat

- [SAT 问题简介 - 中国科学技术大学](http://staff.ustc.edu.cn/~jianmin/lecture/ai2021/sat.pdf)
- [从SAT到SMT——逻辑约束求解研究获新突破----中国科学院软件研究所](http://www.is.cas.cn/xwdt2016/kyjz2016/202208/t20220831_6507435.html)

In [1]:
from ortools.sat.python import cp_model

cp_model.CpModel?

> class CpSolver
Main solver class.
The purpose of this class is to search for a solution to the model provided to the Solve() method.
Once Solve() is called, this class allows inspecting the solution found with the Value() and BooleanValue() methods, as well as general statistics about the solve procedure.

cp_model.CpSolver?

> class CpModel
Methods for building a CP model.

Methods beginning with:

New create integer, boolean, or interval variables.
Add create new constraints and add them to the model.

In [63]:
# 在求解上稍微有点不一样, model并没有solver()
model = cp_model.CpModel()

# 需要创建一个求解器的实例
solver = cp_model.CpSolver()

In [64]:
variables = {}
for r in range(N):
    for c in range(N):
        variables[r, c] = cell if (cell := matrix[r][c]) != 0 else model.NewIntVar(1, N, f'v[{r}{c}]')

model.AddAllDifferent?

Signature: model.AddAllDifferent(*expressions)
Docstring:
Adds AllDifferent(expressions).

This constraint forces all expressions to have different values.

def AddAllDifferent(self, variables)
Adds AllDifferent(variables).

This constraint forces all variables to have different values.

Args
variables

**a list of integer variables.**

Returns
An instance of the Constraint class.

In [65]:
for r in range(N):
    model.AddAllDifferent((variables[r, c] for c in range(N)))

for c in range(N):
    model.AddAllDifferent((variables[r, c] for r in range(N)))

In [66]:
for row in range(n):
    for col in range(n):
        model.AddAllDifferent((variables[r + n * row, c + n * col] for r in range(n) for c in range(n)))

In [67]:
status = solver.Solve(model)

solver.Value?

Signature:
solver.Value(
    expression: Union[ForwardRef('LinearExpr'), ForwardRef('IntVar'), numbers.Integral, numpy.integer, int],
) -> int
Docstring: Returns the value of a linear expression after solve.

In [68]:
if status == cp_model.OPTIMAL:
    print('sudoku solution:')
    # 预先构造一个数独容器用于存放解析结果
    solution = [[0 for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            solution[i][j] = int(solver.Value(variables[i, j]))
    # 绘制数独结果
    sudo_data = (''.join(f'{"| " if c % 3 == 0 else " "}{solution[r][c]}{" " if matrix[r][c] == 0 else "*"}' for c in range(N)) + '|' + ('\n'+ '-' * 31 if (r + 1) % 3 == 0 and r != N - 1 else '') for r in range(N))
    print('\n'.join(sudo_data))
else: print('warning: no solution')

sudoku solution:
| 8* 1  2 | 7  5  3 | 6  4  9 |
| 9  4  3*| 6* 8  2 | 1  7  5 |
| 6  7* 5 | 4  9* 1 | 2* 8  3 |
-------------------------------
| 1  5* 4 | 2  3  7*| 8  9  6 |
| 3  6  9 | 8  4* 5*| 7* 2  1 |
| 2  8  7 | 1* 6  9 | 5  3* 4 |
-------------------------------
| 5  2  1*| 9  7  4 | 3  6* 8*|
| 4  3  8*| 5* 2  6 | 9  1* 7 |
| 7  9* 6 | 3  1  8 | 4* 5  2 |
